In [ ]:
from typing import Tuple
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.sans-serif": "Times",
        "font.size": 10,
    }
)

PAPER_COL_WIDTH_IN = (6.75 - 0.25) / 2
PAPER_CONTENT_WIDTH_IN = 6.75
PAPER_COL_WIDTH_CM = PAPER_COL_WIDTH_IN * 2.54
PAPER_CONTENT_WIDTH_CM = PAPER_CONTENT_WIDTH_IN * 2.54

We assume the simple linear system $y(k) = 1 + u^2(k)$

In [ ]:
u = np.linspace(-5, 5)
def system(u):
    return 1 + u**2

def generate_system_plot(size=(PAPER_COL_WIDTH_IN,PAPER_COL_WIDTH_IN), fig_ax=None) -> Tuple[plt.Figure, mpl.axes.Axes]:
    if fig_ax is None:
        fig, ax = plt.subplots(1,1, figsize=size)
    else: 
        fig, ax = fig_ax
        
    # ax.plot(u, system(u), c="k", label=r"$y(k) = 1 + u^2(k)$")
    ax.plot(u, system(u), c="k", label=r"Nonlinear System")
    ax.axhline(y=0, color="k", alpha=0.25)
    ax.axvline(x=0, color="k", alpha=0.25)
    ax.set_ylim(-5,25)
    ax.set_xlabel(r"$u_\textrm{f}(k)$")
    ax.set_ylabel(r"$y_\textrm{f}(k)$")
    ax.set_box_aspect(1)
    fig.tight_layout()
    ax.tick_params(
        "y",
        which="both",  # both major and minor ticks are affected
        left=False,  # ticks along the bottom edge are off
        labelleft=False,
    )
    ax.tick_params(
        "x",
        which="both",  # both major and minor ticks are affected
        bottom=False,  # ticks along the bottom edge are off
        labelbottom=False,
    )
    return fig, ax

fig, ax = generate_system_plot()
ax.legend()

In [ ]:
def linear_least_square(X, y):
    """Compute Xw = y
    
    where X has (n_pts, n_features)
    """
    w, _, _, _ = np.linalg.lstsq(X, y, rcond=-1)
    return w.squeeze()

def affine_least_square(X, y):
    """Compute Xw + w_0 = y
    
    where X has (n_pts, n_features)
    """
    X = np.append(X, np.ones((X.shape[0],1)), axis=1)
    w, _, _, _ = np.linalg.lstsq(X, y, rcond=-1)
    w_lin = w[0,0]
    w_aff = w[1, 0]
    return w_lin, w_aff

In [ ]:
affine_data_pts_u = np.array([3, 3.5]).reshape(-1,1)
affine_data_pts_y = system(affine_data_pts_u)

u_pred_plot = np.linspace(-1,8)

fig_aff, ax_aff = generate_system_plot((2.5,2.5))
w_lin = linear_least_square(affine_data_pts_u, affine_data_pts_y)

ax_aff.scatter(affine_data_pts_u, affine_data_pts_y, c="red")
ax_aff.plot(u_pred_plot, u_pred_plot * w_lin, label="Linear fit")

w_aff = affine_least_square(affine_data_pts_u, affine_data_pts_y)
print(w_aff)
ax_aff.plot(u_pred_plot, u_pred_plot * w_aff[0] + w_aff[1], label="Affine fit")
ax_aff.legend(loc="lower left")
ax_aff.set_title(r"Linearization vs `Affinization'")
ax_aff.set_xlim(-6,6)
ax_aff.set_ylim(-20,25)
# fig.tight_layout()
fig_aff.savefig("figures/cartoons/lin_vs_aff.pdf", bbox_inches="tight")
fig_aff.show()

In [ ]:
outlier_data_u = np.array([-4, 3, 3.5]).reshape(-1,1)
outlier_data_y = system(outlier_data_u)

w_aff = affine_least_square(outlier_data_u, outlier_data_y)

fig,ax = generate_system_plot((2.5,2.5))
ax.scatter(outlier_data_u, outlier_data_y, c="red")
ax.plot(u, u*w_aff[0]+w_aff[1], label="W/o data selection")

w_aff = affine_least_square(outlier_data_u[1:], outlier_data_y[1:])

ax.scatter(outlier_data_u, outlier_data_y, c="red")

# Add index numbers to each point
for i, (xi, yi) in enumerate(zip(outlier_data_u, outlier_data_y), 1):
    plt.text(xi-0.25, yi, str(i), fontsize=10, ha="right", va="bottom", color="red")

ax.plot(u, u*w_aff[0]+w_aff[1], label="W/ data selection")
ax.legend(loc="lower left")
ax.set_title("Local vs. Global Linearization")
ax.set_xlim(-6, 6)
ax.set_ylim(-20, 25)
# fig.tight_layout()
fig.savefig("figures/cartoons/data_sel.pdf", bbox_inches="tight")

In [ ]:
fig, ax = generate_system_plot((2.5, 2.5))

import scipy.integrate as integrate
import numpy as np


# Perform the integration
geodesic_distance_long, error = integrate.quad(system, -2, 2)
geodesic_distance_short, error = integrate.quad(system, 2, 2.8136)


data = np.array([2, -2, 2.8136])
data_y = system(data)
ax.set_aspect("equal")
sc = ax.scatter(data, data_y, c=[0, geodesic_distance_long, geodesic_distance_short])
plt.plot(data[[0,1]], data_y[[0,1]], ":", c="r")
plt.plot(data[[0,2]], data_y[[0,2]], ":", c="r")
fig.colorbar(sc, label="Geodesic Distance", shrink=0.8)
ax.set_ylim(1, 10)
ax.set_box_aspect(1)
ax.set_title("Geodesic Distance vs\nEuclidean Distance")
fig.savefig("figures/cartoons/geodesic.pdf", bbox_inches="tight")
fig.show()

In [ ]:
from sklearn.manifold import Isomap

isomap_embedder = Isomap(n_neighbors=5, n_components=1)

isomap_data = np.hstack((u.reshape(-1,1), system(u).reshape(-1,1)))
# print(isomap_data.shape)
data_transformed = isomap_embedder.fit_transform(isomap_data)

fig, axs = plt.subplots(1,2, figsize=(5,2.5))

axs[0].scatter(u, system(u), c=data_transformed)
# NOTE(@naefjo): Added a minus so colors are on the same side.
axs[1].scatter(-data_transformed, np.zeros_like(data_transformed), c=data_transformed)
axs[0].set_xlabel(r"$u(k)$")
axs[0].set_ylabel(r"$y(k)$")
axs[0].axhline(y=0, color="k", alpha=0.25)
axs[0].axvline(x=0, color="k", alpha=0.25)
axs[0].set_ylim(-5,25)
axs[1].set_xlabel("Embedding Dimension")
axs[1].tick_params(
    "y",
    which="both",  # both major and minor ticks are affected
    left=False,  # ticks along the bottom edge are off
    labelleft=False,
)
axs[0].set_title("Simple System in 2D")
axs[1].set_title("Corresponding Embedding")
fig.suptitle("Isomap Embedding of the Simple System")
fig.tight_layout()
fig.savefig("figures/cartoons/iso_embedding.pdf", bbox_inches="tight")

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axs = plt.subplots(1,3, figsize=(PAPER_CONTENT_WIDTH_IN, 2.5))

fig, axs[0] = generate_system_plot(fig_ax=(fig, axs[0]))
fig, axs[1] = generate_system_plot(fig_ax=(fig, axs[1]))
fig, axs[2] = generate_system_plot(fig_ax=(fig, axs[2]))
axs[0].scatter(affine_data_pts_u, affine_data_pts_y, c="red")
axs[0].plot(u_pred_plot, u_pred_plot * w_lin, label="Linear fit")

w_aff = affine_least_square(affine_data_pts_u, affine_data_pts_y)
print(w_aff)
axs[0].plot(u_pred_plot, u_pred_plot * w_aff[0] + w_aff[1], label="Affine fit")
axs[0].legend(loc="lower left")
axs[0].set_title("Linearization\nvs. `Affinization'")
axs[0].set_xlim(-6, 6)
axs[0].set_ylim(-20, 25)

w_aff = affine_least_square(outlier_data_u, outlier_data_y)
axs[1].scatter(outlier_data_u, outlier_data_y, c="red")
axs[1].plot(u, u * w_aff[0] + w_aff[1], label="W/o data selection")

w_aff = affine_least_square(outlier_data_u[1:], outlier_data_y[1:])

axs[1].scatter(outlier_data_u, outlier_data_y, c="red")

# Add index numbers to each point
for i, (xi, yi) in enumerate(zip(outlier_data_u, outlier_data_y), 1):
    axs[1].text(xi - 0.3, yi, str(i), fontsize=10, ha="right", va="bottom", color="red")

axs[1].plot(u, u * w_aff[0] + w_aff[1], label="W/ data selection")
axs[1].legend(loc="lower left")
axs[1].set_title("Local vs. Global\nLinearization")
axs[1].set_xlim(-6, 6)
axs[1].set_ylim(-20, 25)


data = np.array([2, -2, 2.8136])
data_y = system(data)
axs[2].set_aspect("equal")
sc = axs[2].scatter(
    data, data_y, c=[0, geodesic_distance_long, geodesic_distance_short]
)
axs[2].plot(data[[0, 1]], data_y[[0, 1]], ":", c="r")
axs[2].plot(data[[0, 2]], data_y[[0, 2]], ":", c="r")
divider = make_axes_locatable(axs[2])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(
    sc,
    cax=cax,
    orientation="vertical",
    label="Geodesic Distance",
)

# axs[2].add_colorbar(sc, label="Geodesic Distance", shrink=0.8)
axs[2].set_ylim(1, 10)
axs[2].set_title("Euclidean vs. Geodesic\n Distance")
# ax.set_box_aspect(1)
ax.set_title("Geodesic Distance vs\nEuclidean Distance")

fig.savefig("figures/cartoons/combined_plots.pdf", bbox_inches="tight")

In [ ]:
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

img_reacher_env = mpimg.imread("../thesis-report/figures/reacher-env.png")
img_rocket_env = mpimg.imread("../thesis-report/figures/rocket_simulation.png")

fig, axs = plt.subplots(2,2, figsize=(PAPER_COL_WIDTH_IN, PAPER_COL_WIDTH_IN), dpi=1500)

fig.subplots_adjust(wspace=0.05, hspace=0.0)
axs[0,0].imshow(img_reacher_env)
axs[0,0].tick_params(
    "both",
    which="both",  # both major and minor ticks are affected
    left=False,  # ticks along the bottom edge are off
    labelleft=False,
    bottom=False,
    labelbottom=False,
)
axs[0,1].imshow(img_rocket_env)
axs[0,1].tick_params(
    "both",
    which="both",  # both major and minor ticks are affected
    left=False,  # ticks along the bottom edge are off
    labelleft=False,
    bottom=False,
    labelbottom=False,
)

axs[0, 0].axis("off")
axs[0, 1].axis("off")

cl_data_sdeepc = np.loadtxt("data/reacher/cl_data/l1_selector/sdeepc_state_100.csv")
cl_data_sdeepc_pred = np.loadtxt("data/reacher/cl_data/l1_selector/sdeepc_predictions_100.csv")
cl_data_deepc = np.loadtxt("data/reacher/cl_data/deepc_state.csv")
cl_data_deepc_pred = np.loadtxt("data/reacher/cl_data/deepc_predictions.csv")
cl_data_streamdeepc = np.loadtxt("data/reacher/cl_data/windowed/deepc_state_100.csv")
cl_data_streamdeepc_pred = np.loadtxt("data/reacher/cl_data/windowed/deepc_predictions_100.csv")

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

line1, = axs[1,0].plot([0, 0.21], [0, 0], "black", linewidth=6, alpha=0.3)
line1.set_solid_capstyle("round")
line2, = axs[1, 0].plot([0, 0.105, 0.0419], [0, 0, 0.09658], "black", linewidth=6, alpha=0.2)
line2.set_solid_capstyle("round")

axs[1,0].plot(cl_data_sdeepc[:,4] + cl_data_sdeepc[:,8], cl_data_sdeepc[:, 5] + cl_data_sdeepc[:,9], label="Select-DeePC (ours)", c=blue)
axs[1,0].plot(cl_data_deepc[:,4] + cl_data_deepc[:,8], cl_data_deepc[:, 5] + cl_data_deepc[:,9], zorder=10, label="Full Data DeePC", c=orange)
axs[1,0].plot(cl_data_streamdeepc[:,4] + cl_data_streamdeepc[:,8], cl_data_streamdeepc[:, 5] + cl_data_streamdeepc[:,9], label="Windowed DeePC", c=green)

for arr1, arr2 in zip(cl_data_sdeepc_pred[4::8],cl_data_sdeepc_pred[5::8]):
    axs[1,0].plot(arr1, arr2, "--", c=blue, alpha=0.5)

skip_factor = 3
for arr1, arr2 in zip(cl_data_deepc_pred[4::skip_factor*8],cl_data_deepc_pred[5::skip_factor*8]):
    axs[1,0].plot(arr1, arr2, "--", c=orange, alpha=0.2)

for arr1, arr2 in zip(cl_data_streamdeepc_pred[4::8],cl_data_streamdeepc_pred[5::8]):
    axs[1, 0].plot(arr1, arr2, "--", c=green, alpha=0.1)

axs[1, 0].scatter([0.043], [0.092], marker="x", c="r", zorder=20, label="Setpoint")

axs[1, 0].tick_params(
    axis="both",
    which="both",  # both major and minor ticks are affected
    bottom=False,  # ticks along the bottom edge are off
    top=False,  # ticks along the top edge are off
    left=False,
    right=False,
    labelbottom=False,  # labels along the bottom edge are off)
    labelleft=False,
)

expl = plt.imread("figures/explosion.png")
expl_img = OffsetImage(expl, zoom=0.04)
rocket = plt.imread("figures/rocket.png")
rocket_img = OffsetImage(rocket, zoom=0.06)
rocket_img_alpha = OffsetImage(rocket, zoom=0.06, alpha=0.5)
axs[1,1].add_artist(AnnotationBbox(expl_img, (34, 13.5), frameon=False))
axs[1,1].add_artist(AnnotationBbox(expl_img, (34, 8), frameon=False))

# fig.legend(loc="lower left")
# axs[1, 0].set_xlabel("$x$")
# axs[1, 0].set_ylabel("$y$")
# axs[1, 0].set_title("Closed Loop\nTrajectories")
# fig.tight_layout()
# fig.savefig("figures/reacher_cl.pdf")


def plot_ol_traj(ol_arr: np.ndarray, axs, color):
    for i in range(0, ol_arr.shape[0], 6):
        plt.plot(ol_arr[i, :], ol_arr[i + 1, :], linestyle="--", alpha=0.5, c=color)


blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

select_traj = np.loadtxt("data/rocket/cl_random_walk/500/state_traj.csv")
select_traj_ol = np.loadtxt("data/rocket/cl_random_walk/500/state_traj_ol.csv")
axs[1, 1].plot(
    select_traj[:, 0], select_traj[:, 1], c=blue, zorder=10#, label="Select-DeePC"
)
plot_ol_traj(select_traj_ol, axs[1, 1], blue)

full_traj = np.loadtxt("data/rocket/cl_random_walk/5208/state_traj.csv")
full_traj_ol = np.loadtxt("data/rocket/cl_random_walk/5208/state_traj_ol.csv")
axs[1, 1].plot(full_traj[:, 0], full_traj[:, 1], c=orange)#, label="Full Data DeePC")
plot_ol_traj(full_traj_ol, axs[1, 1], orange)

# select_traj_2 = x_traj_deepc
# select_traj_ol_2 = x_traj_deepc_predictions.transpose((0, 2, 1)).reshape(
#     -1, x_traj_deepc_predictions.shape[1]
# )
select_traj_2 = np.loadtxt("data/rocket/wdeepc_traj.csv")
select_traj_ol_2 = np.loadtxt("data/rocket/wdeepc_traj_ol.csv")
axs[1, 1].plot(
    select_traj_2[:, 0], select_traj_2[:, 1], c=green#, label="Windowed DeePC"
)
plot_ol_traj(select_traj_ol_2, axs[1, 1], green)


axs[1, 1].add_artist(AnnotationBbox(rocket_img_alpha, (7, 24.5), frameon=False, zorder=200))
axs[1, 1].add_artist(AnnotationBbox(rocket_img_alpha, (7, 22.5), frameon=False, zorder=200))
axs[1, 1].add_artist(AnnotationBbox(rocket_img, (7, 20.5), frameon=False, zorder=200))
axs[1, 1].scatter(16.667, 12.48, c="red", marker="x", zorder=20)#, label="Setpoint")

axs[1, 1].tick_params(
    axis="both",
    which="both",  # both major and minor ticks are affected
    bottom=False,  # ticks along the bottom edge are off
    top=False,  # ticks along the top edge are off
    left=False,
    right=False,
    labelbottom=False,  # labels along the bottom edge are off)
    labelleft=False,
)

axs[1,0].set_box_aspect(1)
axs[1,1].set_box_aspect(1)

fig.legend(ncols=2, bbox_to_anchor=(0.96,0.11), fontsize="small")
fig.savefig("figures/paper/title-figure.pdf", bbox_inches="tight")
